# Video Text Extraction

In [15]:
import moviepy.editor as mp
import speech_recognition as sr
video_name = 'data/test_video.mp4'
output_radio_name = 'data/output_radio.wav'
#output_radio_name = 'data/harvard.wav'
output_radio_chunk = 'data/output_radio_chunk'
output_text_name = 'data/output_text.txt'
WIT_AI_KEY = "5NDLZ4VT4XTWAJNPCBHIMOLAABG2G67M"  # Wit.ai keys are 32-character uppercase alphanumeric strings

### convert video to audio (only run once)

In [3]:

video = mp.VideoFileClip(video_name) 
audio = video.audio
audio.write_audiofile(output_radio_name)

MoviePy - Writing audio in data/output_radio.wav


MoviePy - Done.


#### handle big radio

In [4]:
import os 
from pydub import AudioSegment 
from pydub.utils import make_chunks
from pydub.silence import split_on_silence 
import shutil
  
# a function that splits the audio file into chunks 
# and applies speech recognition 
def silence_based_conversion(input_radio_path = None, output_radio_path = None): 
  
    # open the audio file stored in 
    # the local system as a wav file. 
    audio = AudioSegment.from_wav(input_radio_path) 
  
    # open a file where we will concatenate   
    # and store the recognized text 
    #fh = open("recognized.txt", "w+") 
          
    # split track where silence is 0.5 seconds  
    # or more and get chunks 
    #     chunks = split_on_silence(song, 
    #         # must be silent for at least 0.5 seconds 
    #         # or 500 ms. adjust this value based on user 
    #         # requirement. if the speaker stays silent for  
    #         # longer, increase this value. else, decrease it. 
    #         min_silence_len = 300, 

    #         # consider it silent if quieter than -16 dBFS 
    #         # adjust this per requirement 
    #         silence_thresh = -16,
    #         keep_silence=200
    #     ) 
    chunk_s = 20000
    chunks = make_chunks(audio, chunk_s)
    # create a directory to store the audio chunks. 
    try: 
        os.mkdir(output_radio_path) 
    except(FileExistsError): 
        
        shutil.rmtree(output_radio_path)
        os.mkdir(output_radio_path)
  
    # # move into the directory to 
    # # store the audio files. 
    # os.chdir(output_radio_path) 
  
   
    # process each chunk 
    for i, chunk in enumerate(chunks): 
              
        # Create 0.5 seconds silence chunk 
        #chunk_silent = AudioSegment.silent(duration = 500) 
  
        # add 0.5 sec silence to beginning and  
        # end of audio chunk. This is done so that 
        # it doesn't seem abruptly sliced. 
        #audio_chunk = chunk_silent + chunk + chunk_silent 
  
        # export audio chunk and save it in  
        # the current directory. 
        print("saving chunk{}.wav".format(i)) 
        # specify the bitrate to be 192 k bitrate ='192k'
        chunk.export(output_radio_chunk + "/chunk{}.wav".format(i), format ="wav") 
  
        # the name of the newly created chunk 
        filename = 'chunk'+str(i)+'.wav'
  
        print("Processing chunk "+str(i)) 

In [5]:
silence_based_conversion(output_radio_name, output_radio_chunk)

saving chunk0.wav
Processing chunk 0
saving chunk1.wav
Processing chunk 1
saving chunk2.wav
Processing chunk 2
saving chunk3.wav
Processing chunk 3
saving chunk4.wav
Processing chunk 4
saving chunk5.wav
Processing chunk 5
saving chunk6.wav
Processing chunk 6
saving chunk7.wav
Processing chunk 7
saving chunk8.wav
Processing chunk 8
saving chunk9.wav
Processing chunk 9
saving chunk10.wav
Processing chunk 10
saving chunk11.wav
Processing chunk 11
saving chunk12.wav
Processing chunk 12
saving chunk13.wav
Processing chunk 13
saving chunk14.wav
Processing chunk 14
saving chunk15.wav
Processing chunk 15
saving chunk16.wav
Processing chunk 16
saving chunk17.wav
Processing chunk 17
saving chunk18.wav
Processing chunk 18
saving chunk19.wav
Processing chunk 19
saving chunk20.wav
Processing chunk 20
saving chunk21.wav
Processing chunk 21
saving chunk22.wav
Processing chunk 22
saving chunk23.wav
Processing chunk 23
saving chunk24.wav
Processing chunk 24
saving chunk25.wav
Processing chunk 25
saving

### Load audio and convert speeech to text (small radio)

In [7]:
# Initialize recognizer 
r = sr.Recognizer() 

# Load the audio file 

with sr.AudioFile(output_radio_name) as source: 
    data = r.record(source) 
  
# Convert speech to text 
#text = r.recognize_sphinx(data)

In [12]:
# recognize speech using sphinx
text = r.recognize_sphinx(data)
print(text)

this they'll smell of old we're lingers it takes heat to bring out the odor they called it restores health and zest case all the gold he's fine with him tacos all pastore my favorite he's as full food is the hot cross bonn


In [11]:
# recognize speech using Wit.ai

try:
    print("Wit.ai thinks you said " + r.recognize_wit(data, key=WIT_AI_KEY))
except sr.UnknownValueError:
    print("Wit.ai could not understand audio")
except sr.RequestError as e:
    print("Could not request results from Wit.ai service; {0}".format(e))

Wit.ai thinks you said The stale smell of old beer lingers. It takes heat to bring out the odor. A cold dip restores health and zest. A salt pickle taste fine with ham. Tacos al pastora are my favorite. A zestful food is the hot cross bun


### Load audio and convert speeech to text (large radio)

In [8]:
import re
from os import listdir
from os.path import isfile, join
output_radio_chunk_file_lst = [f for f in listdir(output_radio_chunk) if isfile(join(output_radio_chunk, f))]
output_radio_chunk_file_lst.sort(key=lambda x: int(re.findall(r'\d+', x)[0]), reverse=False)
if os.path.exists(output_text_name):
    os.remove(output_text_name)
    print('removed existed output file')
with open(output_text_name, "w+") as fh:
    for i, f in enumerate(output_radio_chunk_file_lst):
        with sr.AudioFile('/'.join([output_radio_chunk, f])) as source: 
            data = r.record(source) 
            try:
                tex = r.recognize_wit(data, key=WIT_AI_KEY)
                print(f"chunk{i}: " + tex )
                
                fh.write(tex+". ") 
            except sr.UnknownValueError:
                print("Wit.ai could not understand audio")
            except sr.RequestError as e:
                print("Could not request results from Wit.ai service; {0}".format(e))


chunk0: Welcome to energy 101 where okay here's the deal our name is in question right now but the can the not consensus the theme is not so we ask the dumb question
chunk1: But no question is actually dumb so they're not dumb. So you don't have? That's questionable. We as a basic questions about energy because we want to learn and we're tired of pretending like we know anything about energy. So you are here as our second guest. This is Justin
chunk2: Goatier. Yeah, you got it. Oh my god. Yeah. I wouldn't be the first podcast. You were on that did not talk about your last name for 5 minutes. Thank you. You're welcome. Moving on. I love talking about myself in my last video. It's okay. But no, thank you so much for having me on. Pump you
chunk3: Send me a message and I was like, absolutely, any, you know, anything to support digital wildcatters, just to be involved and at the end of the day, educating hopefully people who have a misconception about energy. That's the goal, right? Yeah. 

### convert speeech to text using whisper (dont need to chunk)
https://github.com/SYSTRAN/faster-whisper

In [17]:
whisper_output_text_name = 'data/whisper_output_text.txt'

In [16]:
from faster_whisper import WhisperModel

model_size = "large-v3"

# Run on GPU with FP16
model = WhisperModel(model_size, device="cuda", compute_type="float16")

# or run on GPU with INT8
# model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
# or run on CPU with INT8
# model = WhisperModel(model_size, device="cpu", compute_type="int8")

segments, info = model.transcribe(
    output_radio_name,
    beam_size=5,
    vad_filter=True,
    vad_parameters=dict(min_silence_duration_ms=500),
)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))
with open(output_text_name, "w+") as fh:
    for segment in segments:
        print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
        fh.write("[%.2fs -> %.2fs] %s \n" % (segment.start, segment.end, segment.text))

Detected language 'en' with probability 1.000000
[0.00s -> 10.70s]  welcome to energy 101 where okay here's the deal our name is in question right now but
[10.70s -> 21.08s]  the consistent can the not consensus the theme is not so we ask the dumb questions but no question
[21.08s -> 24.82s]  is actually dumb so they're not dumb so you don't have that's questionable
[24.82s -> 32.38s]  we ask the basic questions about energy because we want to learn and we're tired of pretending like
[32.38s -> 38.56s]  we know anything about energy um so you are here as our second guest this is
[38.56s -> 46.18s]  justin gauthier yeah you got it oh my god yeah i wanted to be the first podcast you were on
[46.18s -> 52.40s]  that did not talk about your last name for five minutes thank you you're welcome moving on
[52.40s -> 54.84s]  i love talking
[54.86s -> 59.60s]  about myself and my last name oh it's okay uh but no thank you so much for having me on i'm
[59.60s -> 64.22s]  pumped you sent me a mes